<a href="https://colab.research.google.com/github/kyhchn/skripsheesh/blob/main/PENGUJIAN_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: install z3 prover

!pip install z3-solver


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 11.8 MB/s eta 0:00:00


In [ ]:
from z3 import *


# === Step 1: Enum untuk Role, Action, Resource ===
Roles, (Superadmin, Direktur, Finance, Kasir, Resepsionis, Pegawai_Lapangan, Pelanggan) = EnumSort("Role", [
    "Superadmin", "Direktur", "Finance", "Kasir", "Resepsionis", "Pegawai_Lapangan", "Pelanggan"
])

Actions, (R, C, U, D, Impersonate) = EnumSort("Action", ["R", "C", "U", "D", "Impersonate"])

Resources, (
    User, Customer, MeterReading, Bill, Permission, Role, Role_Perm,
    Journal, COA, FinancialReport, HousingArea, NoneResource
) = EnumSort("Resource", [
    "User", "Customer", "MeterReading", "Bill", "Permission", "Role", "Role_Perm",
    "Journal", "COA", "FinancialReport", "HousingArea", "None"
])

# === Step 2: Function untuk permission dan inheritance ===
has_permission = Function("has_permission", Roles, Actions, Resources, BoolSort())
inherits = Function("inherits", Roles, Roles, BoolSort())

# === Step 3: Mapping string ke enum (biar gampang parsing) ===
role_map = {
    "Superadmin": Superadmin, "Direktur": Direktur, "Finance": Finance,
    "Kasir": Kasir, "Resepsionis": Resepsionis, "Pegawai_Lapangan": Pegawai_Lapangan, "Pelanggan": Pelanggan
}
action_map = {"R": R, "C": C, "U": U, "D": D, "Impersonate": Impersonate}
resource_map = {
    "User": User, "Customer": Customer, "MeterReading": MeterReading, "Bill": Bill,
    "Permission": Permission, "Role": Role, "Role_Perm": Role_Perm,
    "Journal": Journal, "COA": COA, "FinancialReport": FinancialReport,
    "HousingArea": HousingArea, "None": NoneResource
}

# === Step 4: Masukkan fakta dari hasil kamu tadi ===
s = Solver()

def add_tracked(expr, name):
    s.assert_and_track(expr, String(name))


def perm_tuple(p):
    act, res = map(str.strip, p.split(","))
    return action_map[act], resource_map.get(res, NoneResource)

role_perms = {
    "Superadmin": [
        "R, User", "C, User", "U, User", "R, Customer", "R, MeterReading", "C, Bill",
        "C, Permission", "R, Permission", "U, Permission", "D, Permission",
        "R, Role", "R, Role_Perm", "C, Role", "C, Role_Perm",
        "U, Role", "D, Role_Perm", "D, Role",
        "Impersonate, Finance", "Impersonate, Kasir", "Impersonate, Pegawai_Lapangan",
        "Impersonate, Resepsionis", "Impersonate, Direktur"
    ],
    "Direktur": ["R, FinancialReport"],
    "Finance": ["C, Journal", "R, Journal", "U, Journal", "C, COA", "R, COA", "U, COA", "R, User"],
    "Kasir": ["U, Bill", "C, Journal", "R, Journal", "R, COA", "U, Journal", "R, User"],
    "Resepsionis": ["C, Customer", "U, Customer", "C, HousingArea", "R, HousingArea", "U, HousingArea", "R, User"],
    "Pegawai_Lapangan": ["C, MeterReading", "U, MeterReading"],
    "Pelanggan": ["R, Customer", "R, MeterReading", "R, Bill"]
}

# Masukkan permission ke dalam Z3
for role, perms in role_perms.items():
    for p in perms:
        act, res = perm_tuple(p)
        s.add(has_permission(role_map[role], act, res))

# === Step 5: Masukkan inheritance ===
inherit_rel = [
    ("Direktur", "Kasir"),
    ("Kasir", "Pelanggan"),
    ("Resepsionis", "Pelanggan"),
    ("Pegawai_Lapangan", "Pelanggan")
]

for parent, child in inherit_rel:
    s.add(inherits(role_map[parent], role_map[child]))

# === Base Case done! ===
print("✅ Base Z3 model siap digunakan!")


✅ Base Z3 model siap digunakan!


In [ ]:
users = [Const(f"user_{i}", Roles) for i in range(13)]

s.push()

# === Step 3: Fakta role assignment ===
s.add(users[0] == Superadmin)        # 1 Superadmin
s.add(users[1] == Direktur)          # 1 Direktur
s.add(users[2] == Finance)           # 2 Finance
s.add(users[3] == Finance)
s.add(users[4] == Kasir)             # 2 Kasir
s.add(users[5] == Kasir)
s.add(users[6] == Resepsionis)       # 2 Resepsionis
s.add(users[7] == Resepsionis)
s.add(users[8] == Pegawai_Lapangan)  # 2 Pegawai Lapangan
s.add(users[9] == Pegawai_Lapangan)
s.add(users[10] == Pelanggan)        # 2 Pelanggan
s.add(users[11] == Pelanggan)
s.add(users[12] == Superadmin)

if s.check() == sat:
    m = s.model()
    print("✅ Fakta role assignment:")
    for i in range(len(users)):
        print(f"User {i+1}: {m[users[i]]}")
else:
    print("❌ Tidak bisa assign role dengan fakta tersebut.")

✅ Fakta role assignment:
User 1: Superadmin
User 2: Direktur
User 3: Finance
User 4: Finance
User 5: Kasir
User 6: Kasir
User 7: Resepsionis
User 8: Resepsionis
User 9: Pegawai_Lapangan
User 10: Pegawai_Lapangan
User 11: Pelanggan
User 12: Pelanggan
User 13: Superadmin


In [ ]:
print("\n SOD_02")
print("\n🔍 Checking constraint: setiap user hanya satu role (valid role)")
s.push()
for u in users:
    s.add(Or(u == Superadmin, u == Direktur, u == Finance, u == Kasir,
             u == Resepsionis, u == Pegawai_Lapangan, u == Pelanggan))
if s.check() == sat:
    print("✅ Passed: Semua user memiliki role yang valid.")
else:
    print("❌ Failed: Ada user dengan role tidak valid.")
s.pop()


🔍 Checking constraint: setiap user hanya satu role (valid role)
✅ Passed: Semua user memiliki role yang valid.


In [ ]:
# === Constraint Debug Check 2: Hanya satu Superadmin ===
print("\n CA_01")
print("\n🔍 Checking constraint: hanya satu Superadmin")
s.push()

count_superadmin = Sum([If(users[i] == Superadmin, 1, 0) for i in range(len(users))])
s.add(count_superadmin == 1)
if s.check() == sat:
    print("✅ Passed: Hanya satu Superadmin.")
else:
    print("❌ Failed: Jumlah Superadmin tidak tepat.")
s.pop()


🔍 Checking constraint: hanya satu Superadmin
❌ Failed: Jumlah Superadmin tidak tepat.


In [ ]:
# === Constraint Debug Check: Tiap role minimal satu user ===
print("\n CA_02")
print("\n🔍 Checking constraint: tiap role minimal satu user")

s.push()
roles = [Superadmin, Direktur, Finance, Kasir, Resepsionis, Pegawai_Lapangan, Pelanggan]

for role in roles:
    count = Sum([If(user == role, 1, 0) for user in users])
    s.add(count >= 1)

if s.check() == sat:
    print("✅ Passed: Semua role dimiliki minimal oleh satu user.")
else:
    print("❌ Failed: Ada role yang tidak dimiliki oleh user mana pun.")
s.pop()


🔍 Checking constraint: tiap role minimal satu user
✅ Passed: Semua role dimiliki minimal oleh satu user.


In [ ]:
# Constraint: Tiap permission minimal digunakan satu role
print("\n CA_03")
print("\n🔍 Checking constraint: tiap permission minimal digunakan satu role")

# Ambil permissions unik yang ada di role_perms
permissions_used = set()
for perms in role_perms.values():
    for p in perms:
        act, res = perm_tuple(p)
        permissions_used.add((act, res))

# Pastikan tiap permission digunakan minimal satu role
for permission in permissions_used:
    s.add(Sum([If(has_permission(role, permission[0], permission[1]), 1, 0) for role in roles]) >= 1)

# Debug: check jika constraint diterima
s.push()
if s.check() == sat:
    print("✅ Passed: Semua permission digunakan minimal satu role.")
else:
    print("❌ Failed: Ada permission yang tidak digunakan oleh role mana pun.")
s.pop()



🔍 Checking constraint: tiap permission minimal digunakan satu role
✅ Passed: Semua permission digunakan minimal satu role.


In [ ]:
# Constraint: Role yang memiliki permission Update harus memiliki permission Read untuk objek terkait
print("\n OP_07")
print("\n🔍 Checking constraint: Role yang memiliki permission Update harus memiliki permission Read untuk objek terkait")

# Iterasi setiap role dan permission untuk memeriksa Update dan Read pada resource yang sama
for role in roles:
    for act, res in permissions_used:
        # Jika role memiliki permission Update untuk resource tertentu, maka harus ada permission Read untuk resource tersebut
        s.add(Implies(
            has_permission(role, U, res),  # Jika role memiliki Update pada resource res
            has_permission(role, R, res)   # Maka role harus memiliki Read pada resource res
        ))

# Debug: check jika constraint diterima
s.push()
if s.check() == sat:
    print("✅ Passed: Role yang memiliki permission Update juga memiliki permission Read untuk objek terkait.")
else:
    print("❌ Failed: Ada role yang tidak memenuhi constraint Update harus punya Read untuk objek terkait.")
s.pop()



🔍 Checking constraint: Role yang memiliki permission Update harus memiliki permission Read untuk objek terkait
✅ Passed: Role yang memiliki permission Update juga memiliki permission Read untuk objek terkait.



 OP_08
🔍 Checking constraint: Tidak boleh ada role apa pun yang memiliki ketiga permission {C, MeterReading}, {C, Bill}, dan {C, Journal} secara bersamaan


NameError: name 'roles' is not defined